In [3]:
import pandas as pd
from pymongo import MongoClient
import requests
import json

# API

From dataframe to dict

In [4]:
data = pd.read_csv('../data/clean_data.csv', index_col='Unnamed: 0')

In [71]:
data.head(30)

,Brand,Category,Name,Ingredients,URL,Price,Rating
0,Algenist,Face Serum,GENIUS Liquid Collagen,-Patented Alguronic Acid: Naturally sourced a...,https://www.sephora.com/product/genius-liquid-...,95.45,4.0
1,Algenist,Moisturizer,GENIUS Sleeping Collagen,-Patented Alguronic Acid: Naturally sourced a...,https://www.sephora.com/product/genius-sleepin...,81.34,4.5
2,Algenist,Moisturizer,GENIUS Ultimate Anti-Aging Cream,-Patented Alguronic Acid: Naturally sourced a...,https://www.sephora.com/product/genius-ultimat...,92.96,4.5
3,Algenist,Eye Cream,Complete Eye Renewal Balm,-Patented Alguronic Acid: Visibly minimizes t...,https://www.sephora.com/product/complete-eye-r...,56.44,4.0
4,Algenist,Sunscreen & After Sun Care,SUBLIME DEFENSE Ultra Lightweight UV Defense F...,-Patented Alguronic Acid: Helps minimize the ...,https://www.sephora.com/product/sublime-defens...,23.24,4.5
5,Algenist,Eye Cream,GENIUS Ultimate Anti-Aging Eye Cream,-Alguronic Acid: Combats fine lines and wrink...,https://www.sephora.com/product/genius-ultimat...,61.42,4.0
6,Algenist,Face Serum,GENIUS Ultimate Anti-Aging Vitamin C+ Serum,-Alguronic Acid: Improves the appearance of fi...,https://www.sephora.com/product/genius-ultimat...,97.94,4.0
7,Algenist,Face Mask,ALIVE Prebiotic Balancing Mask,-Patented Alguronic Acid: Clinically proven t...,https://www.sephora.com/product/alive-prebioti...,31.54,4.5
8,Algenist,Moisturizer,Regenerative Anti-Aging Moisturizer,-Alguronic Acid: Increases cell regeneration ...,https://www.sephora.com/product/regenerative-a...,78.02,4.5
9,Algenist,Moisturizer,Overnight Restorative Cream,-Alguronic Acid: Supports natural cell regene...,https://www.sephora.com/product/overnight-rest...,78.02,4.5


In [5]:
data_dict = data.to_dict('records')

In [6]:
data_json = json.dumps(data_dict)

In [10]:
with open('skincare_data.json',  'w') as json_file:
    json.dump(data_dict, json_file)

## Mongo queries

For example:

In [13]:
con = MongoClient("localhost:27017")

In [15]:
db = con.get_database("skincare")

In [16]:
collection = db.get_collection("first")

In [18]:
one = list(collection.find({'Brand':'Anthony'}, {'_id':0}))

In [19]:
one

[{'Brand': 'Anthony',
  'Category': 'Cleanser',
  'Name': 'Glycolic Facial Cleanser',
  'Ingredients': ' -Glycolic Acid (4.9%): Works to loosen the surface layer of dead cells and helps clear oil.-Aloe Vera: Moisturizes- soothes- and promotes healing while calming skin without creating a greasy feeling.-Vitamin A: Provides antioxidant action to soften- smooth- and increase elasticity of skin.-Vitamin C: Provides antioxidant action to protect- firm- and brighten skin.\n\n\n\nWater- Cocoamidopropyl Oxide- Glycerin- Cetyl Alcohol- Stearyl Alcohol- Potassium Hydroxide- Sodium Phytate- Citrus Grandis Peel Oil- Citrus Noblis Peel Oil- Ascorbic Acid- Tocopheryl Acetate- Retinyl Acetate- Methylchloroisothiazolinone- Methylisothiazolione- Calendula Officinalis Flower Extract- Anthemis Noblis Flower Extract.',
  'URL': 'https://www.sephora.com/product/glycolic-facial-cleanser-P9038?icid2=products grid:p9038',
  'Price': 24.9,
  'Rating': 4.5},
 {'Brand': 'Anthony',
  'Category': 'Face Set',
  'N

In [30]:
user_input = input('Please choose a brand:')

Please choose a brand:Clarins


In [33]:
two = list(collection.find({'Brand':user_input}, {'_id':0}).limit(5))

In [34]:
two

[{'Brand': 'Clarins',
  'Category': 'Face Serum',
  'Name': 'Double Serum Complete Age Control Concentrate',
  'Ingredients': ' -Turmeric: Contains exceptional anti-aging properties.-Oat Sugars: Visibly firm and lift.\n-Orthosiphon Extract: Helps improve the skin texture.-Callicarpa Extract: Revives radiance.Water- Cetearyl Isononanoate- Glycerin- Isononyl Isononanoate- Caprylic/Capric Triglyceride- Pentylene Glycol- Ppg-3 Myristyl Ether- Ethylene/Propylene/Styrene Copolymer. Dipsacus Sylvestris Extract- Butylene Glycol- Parfum/ Fragrance-Phenoxyethanol-Propanediol-Tromethamine. Silybum Marianum Seed Oil- Carbomer-Tocopheryl Acetate- Escin- Cheno -\nPodium Quinoa Seed Extract- Ethylhexylglycerin- Squalane-Avena Sativa (Oat) Kernel Extract- Theobroma Cacao (Cocoa) Extract- Butylene/Ethylene/Styrene Copolymer- Leontopodium Alpinum Extract- Xanthan Gum- Persea Gratissima (Avocado) Oil Unsaponifiables-Caramel-Curcuma Longa (Turmeric) Root Extract-Salicyloyl Phytosphingosine- Musa Sapientum

For API:

In [64]:
# Si me pasan un input  en forma de string separado por comas

inputs = input('Please choose a brand:')

Please choose a brand:boscia, Clarins


In [65]:
# Lo paso a lista 

inputs_list = inputs.split(',')

In [66]:
inputs_list

['boscia', ' Clarins']

In [67]:
#  Query

three = list(collection.find({'Brand': {'$in':inputs_list}}, {'_id':0}))

In [68]:
three

[{'Brand': 'boscia',
  'Category': 'Cleanser',
  'Name': 'Clear Complexion Cleanser',
  'Ingredients': ' -Willow Bark Extract: A natural Beta Hydroxy Acid (Salicylic Acid) derived from the Willow Tree that gently and effectively exfoliates to unclog and purify pores- as well as refining their appearance.-Rosemary Leaf Extract: An effective antioxidant and anti-inflammatory that helps to visibly firm skin and reduce the appearance of pores.\t-Geranium: Supports circulation- balances the production of sebum- and helps tighten skin’s appearance.  Water- Hydrogenated Starch Hydrolysate- Disodium Cocoyl Glutamate- Diglycerin- Acrylates/Steareth-20 Methacrylate Copolymer- Sodium Cocoyl Glycinate- Butylene Glycol- Methyl Gluceth-10- Sodium Cocoyl Glutamate- Salix Nigra (Willow) Bark Extract- Simmondsia Chinensis (Jojoba) Seed Extract- Epilobium Angustifolium/Leaf/Stem Extract- Rosmarinus Officinalis (Rosemary) Leaf Extract- Pelargonium Graveolens Flower Oil- Citric Acid- 1-2-Hexanediol- Capry

In [70]:
five = []
for i in inputs_list:
    four = list(collection.find({'Brand':i}, {'_id':0}).limit(5))
    five.append(four)
print(five)

[[{'Brand': 'boscia', 'Category': 'Cleanser', 'Name': 'Clear Complexion Cleanser', 'Ingredients': ' -Willow Bark Extract: A natural Beta Hydroxy Acid (Salicylic Acid) derived from the Willow Tree that gently and effectively exfoliates to unclog and purify pores- as well as refining their appearance.-Rosemary Leaf Extract: An effective antioxidant and anti-inflammatory that helps to visibly firm skin and reduce the appearance of pores.\t-Geranium: Supports circulation- balances the production of sebum- and helps tighten skin’s appearance.  Water- Hydrogenated Starch Hydrolysate- Disodium Cocoyl Glutamate- Diglycerin- Acrylates/Steareth-20 Methacrylate Copolymer- Sodium Cocoyl Glycinate- Butylene Glycol- Methyl Gluceth-10- Sodium Cocoyl Glutamate- Salix Nigra (Willow) Bark Extract- Simmondsia Chinensis (Jojoba) Seed Extract- Epilobium Angustifolium/Leaf/Stem Extract- Rosmarinus Officinalis (Rosemary) Leaf Extract- Pelargonium Graveolens Flower Oil- Citric Acid- 1-2-Hexanediol- Caprylyl G